In [ ]:
pip install Biopython

In [ ]:
# import packages
import os
import regex as re
from Bio.Seq import MutableSeq
from Bio.Seq import reverse_complement
from Bio import SeqIO

In [ ]:
def guide_locator(gRNA_len,target):
    #indexes starting pos of all PAM (NGG) sequences and then counts length of guide RNA upstream to locate starting pos of guides
    PAM_lo = [m.start(0) for m in re.finditer("[ATGC]GG",target, overlapped= True)]
    glo = [x - gRNA_len for x in PAM_lo] 
    PAM_lo
    glo

    # finds all possible guides
    poss_guides = []
    for i in range(len(PAM_lo)):
        if glo[i] >= 0:  #this is incase the PAM is too close to the beginning(guideRNA would be less than set length)
            poss_guides.append(target[glo[i]:PAM_lo[i]])
    return poss_guides


In [ ]:
# START
# Load RepeatMasker annotations
with open("/home/milanamilan/design1_copy/sgRNA_design/repeatmasker.out", "r") as f:
    repeat_seq = f.readlines()

In [ ]:
def fix_find(fasta_file,target_te,gRNA_len,minGC,maxGC,aligner):
    
    #maybe just add genome as input for this function and then parse w the parsing genome code after

    # Load and Parse human genome
    from Bio import SeqIO

    # Parse the FASTA file and store the records in a list
    records_dict = SeqIO.to_dict(SeqIO.parse(fasta_file, "fasta"))

    
    ##############

    #Find target sequences
    targets = []

    for line in repeat_seq[3:20]:
        fields = line.strip().split()

        for single_TE in target_te:

            if len(fields) > 13  and fields[9] == single_TE:
                start = int(fields[5])
                end = int(fields[6])
                strand = fields[8]

                for record_id, record in records_dict.items():
                    te_seq = (record.seq[start:end])
                    if strand == "-":
                        te_seq = te_seq.reverse_complement()
                if te_seq != '':
                    targets.append(str(te_seq)) if str(te_seq) not in targets else None #tabbing this line 2x makes the difference between 1 seq or 705

    #print (len(te_seq))
    #print (end-start)
    #print (te_seq)
    #print (len(targets))

    
    ##########


    for te in targets: 

        #Reverse-complement sequence
        rev_comp_seq = reverse_complement(te)
        (rev_comp_seq)

        #locate guides in both strands
        sense_guides= guide_locator(gRNA_len,target=te)
        anti_guides = guide_locator(gRNA_len,target=rev_comp_seq)

    #combine guides into one list
    unfiltered_guides = sense_guides + anti_guides
    
    #filters guides based on GC content
    x= GC_monitor(unfiltered_guides,minGC,maxGC)

    # Write filtered guides to text file for CrispRitz Aligner
    if aligner == "ritz":
        for guide in range((len)(x)):
            x[guide] += "NNN"

        with open('EMX1.sgRNA.txt', 'w') as file:
                for guide in x:
                    file.write(guide + "\n")

    #Write guides to FASTA file for bowtie                
    if aligner == "bowtie":
     with open('input_guides.fasta', 'w') as input_file:
        for i, seq in enumerate(x,1):
            input_file.write(f'>guide_RNA_{i}\n{seq}\n')
    
    return targets

In [ ]:
#this is the fix it code
fix_find("/home/milanamilan/design1_copy/sgRNA_design/hg38_chr19.fna",["LTR2B","MIR"],20,.4,.8,"ritz")

In [ ]:
#result should be 6 different sequences